### DATA

需要分别读入face和pose的数据并对应起来
- 对于siblings_face 需要读入json  
- 对于siblings_pose 需要读入csv    
- 对于Label 需要读入csv


*特别注意对于face，需要去除置信度小于阈值的数据  
*要将video的label扩展到每个frame

In [1]:
import csv
import pandas as pd
import numpy as np
import json

In [2]:
import os
import pandas as pd

# 读取标签文件
label = pd.read_csv('labelA/回应情况-表格 1.csv')

In [13]:
label.head()


,切片ID,回应情况,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17
0,NaN,强,中,弱,没有回应（忽视）,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,15YS_20230317_01/VCAM_0000,0,0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,15YS_20230317_01/VCAM_0000_1,0,0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,15YS_20230317_01/VCAM_0000_2,1,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,15YS_20230317_01/VCAM_0000_3,0,0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
label.iloc[1,0][-4:]

'0000'

导入face特征

In [4]:
# set confidence threshold
confidence_threshold = 0.5

# 导入face特征并对齐标签
features = []
labels = []
Mcount = 0
Dcount = 0
for i in range(1,len(label)):
    directory = label.iloc[i, 0]
    # cam = directory.split('/VCAM_')[-1]

    label_value = label.iloc[i, 1:5] # 取1-4位的标签
    feature_file = os.path.join('siblings_face', directory + '.csv')
    try:
        feature_data = pd.read_csv(feature_file)

        # 丢弃置信度小于0.8的数据
        feature_data = feature_data[feature_data['confidence'] >= confidence_threshold]
        
        # 每一frame加入
        features.append(feature_data.values) 

        # 重复frame标签
        for j in range(feature_data.shape[0]):
            labels.append(label_value.astype("int"))
        
        Dcount += 1
    except:
        print(f"error: {feature_file} not exist")
        Mcount +=1
        continue
    

print(f"Import {Dcount} files, missing {Mcount} files")

# 将特征和标签转换为numpy数组
X = np.concatenate(features, axis=0)
y = np.array(labels)

print(f"X:{len(X)}, y:{len(y)}")


error: siblings_face/15YS_20230317_01/VCAM_0000_3.csv not exist
error: siblings_face/15YS_20230317_01/VCAM_0000_37.csv not exist
error: siblings_face/15YS_20230317_01/VCAM_0001_29.csv not exist
error: siblings_face/15YS_20230317_01/VCAM_0001_30.csv not exist
error: siblings_face/15YS_20230317_01/VCAM_0001_37.csv not exist
error: siblings_face/15YS_20230317_01/VCAM_0002_53.csv not exist
error: siblings_face/15YS_20230317_01/VCAM_0002_76.csv not exist
error: siblings_face/15YS_20230317_01/VCAM_0003_89.csv not exist
error: siblings_face/16YS_20230317_01/VCAM_0004_126.csv not exist
error: siblings_face/16YS_20230317_01/VCAM_0006_18.csv not exist
error: siblings_face/16YS_20230317_01/VCAM_0006_81.csv not exist
error: siblings_face/16YS_20230317_01/VCAM_0006_103.csv not exist
Import 857 files, missing 12 files
X:19325, y:19325


In [ ]:
0.8 # 16000
0.7 # 17393
0.6 # 18295
0.5 # 19325

导入pose特征

In [ ]:
import json

# 导入特征并对齐标签
features = []
labels = []
Mcount = 0
Dcount = 0

# for a single video
for i in range(1,len(label)):
    directory = label.iloc[i, 0] 
    # VCAM number
    Date = directory.split('/')[0] # 15YS_20230317_01
    Vcam = directory.split('/')[-1]  # VCAM_xxxx_xx
    VcamID = Vcam.split('_')[1] # xxxx

    # reconstruct the correct directory
    video_directory = os.path.join('siblings_pose', Date,'VCAM_'+VcamID, Vcam)
    json_directory = os.path.join(video_directory, 'json')

    # label
    label_value = label.iloc[i, 1:5] # 取1-4位的标签

    json_files = os.listdir(json_directory)
    json_files.sort(key=lambda x: int(x.split('_')[2])) # sort by the frame id
    pose_json_data = []

    # for a single frame
    for file in json_files: # for a single frame
        file_path = os.path.join(json_directory, file)
        try:
            with open(file_path, 'r') as f:
                data = json.load(f)
            
            # 漏检测 只有1人的情况
            if len(data['people']) != 2:
                row0 = data['people'][0]['pose_keypoints_2d']
                # 插入face_id
                row0.insert(0,0)
                 # 插入frame
                frame_id = int(file.split('_')[-2])//5 +1 # 恢复frame id
                row0.insert(0,frame_id)
                pose_json_data.append(row0)

                continue
            
            row0 = data['people'][0]['pose_keypoints_2d']
            row1 = data['people'][1]['pose_keypoints_2d']


            # face_id对应 x较小 标记为1，x较大 标记为0 
            if(row0[0]<row1[0]):
                row0,row1 = row1,row0 # swap

            # 插入face_id
            row0.insert(0,0)
            row1.insert(0,1)

            # 插入frame
            frame_id = int(file.split('_')[-2])//5 # 恢复frame id
            row0.insert(0,frame_id)
            row1.insert(0,frame_id)


            # 重复frame标签
            # labels.append(label_value.astype("int"))
            pose_json_data.append(row0)
            pose_json_data.append(row1)

        except:
            print(f"pose error: {file} not exist")
            continue

    features.append(np.array(pose_json_data))
    

# 将特征和标签转换为numpy数组
X = np.concatenate(features, axis=0)
y = np.array(labels)

print(f"Import {Dcount} files, missing {Mcount} files")
print(f"X:{len(X)}, y:{len(y)}")

In [45]:
X[0]

array([ 1.00000e+00,  0.00000e+00,  0.00000e+00,  6.20000e-01,
        0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00,
        0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00,
        0.00000e+00,  1.41040e+03,  1.41190e+03,  1.41540e+03,
        1.41880e+03,  1.42020e+03,  1.41890e+03,  1.41510e+03,
        1.41170e+03,  1.40630e+03,  1.40810e+03,  1.41080e+03,
        1.41450e+03,  1.41830e+03,  1.42140e+03,  1.42370e+03,
        1.42110e+03,  1.41770e+03,  1.41410e+03,  1.41080e+03,
        1.40810e+03,  1.41380e+03,  1.41530e+03,  1.41690e+03,
        1.41750e+03,  1.41700e+03,  1.41550e+03,  1.41390e+03,
        1.41330e+03,  1.45630e+03,  1.45760e+03,  1.46140e+03,
        1.46530e+03,  1.46720e+03,  1.46580e+03,  1.46210e+03,
        1.45780e+03,  1.45270e+03,  1.45500e+03,  1.45820e+03,
        1.46240e+03,  1.46650e+03,  1.46980e+03,  1.47220e+03,
        1.47000e+03,  1.46680e+03,  1.46300e+03,  1.45910e+03,
        1.45540e+03,  1.46010e+03,  1.46180e+03,  1.463

In [10]:
y[0]

array([0, 0, 1, 0])

### 合并提取

In [17]:
# set openface confidence threshold
confidence_threshold = 0.5

# 导入特征并对齐标签
features = []
features_face = []
features_pose = []
labels = []

for i in range(1,len(label)):
    directory = label.iloc[i, 0]
    # VCAM number
    Date = directory.split('/')[0] # 15YS_20230317_01
    Vcam = directory.split('/')[-1]  # VCAM_xxxx_xx
    VcamID = Vcam.split('_')[1] # xxxx

    # reconstruct the correct Openpose directory
    video_directory = os.path.join('siblings_pose', Date,'VCAM_'+VcamID, Vcam)
    json_directory = os.path.join(video_directory, 'json')

    json_files = os.listdir(json_directory)
    json_files.sort(key=lambda x: int(x.split('_')[2])) # sort by the frame id
    

    # the correct Openface csv file
    feature_file = os.path.join('siblings_face', directory + '.csv')

    # the label data
    label_value = label.iloc[i, 1:5] # 取1-4位的标签

    # Load face data of a video
    try:
        face_csv_data = pd.read_csv(feature_file)

        # 丢弃置信度小于0.5的数据
        # face_csv_data = face_csv_data[face_csv_data['confidence'] >= confidence_threshold]

        # 找到frame中有多于2个不同face_id的帧
        counts = face_csv_data.groupby('frame')['face_id'].nunique()
        frames_to_drop = counts[counts > 2].index.tolist()

        # 对于每个需要处理的帧，删除置信度最低的行，直到只剩下2个face_id
        for frame in frames_to_drop:
            frame_df = face_csv_data[face_csv_data['frame'] == frame]
            while frame_df['face_id'].nunique() > 2:
                min_confidence = frame_df['confidence'].min()
                rows_to_drop = frame_df[(frame_df['confidence'] == min_confidence)]['face_id'].tolist()
                frame_df = frame_df[~frame_df['face_id'].isin(rows_to_drop)]
            face_csv_data = face_csv_data[face_csv_data['frame'] != frame]
            face_csv_data = pd.concat([face_csv_data, frame_df], ignore_index=True)


        # # 每一frame加入
        features_face.append(face_csv_data.values) 

        # 重复frame标签
        # for j in range(face_csv_data.shape[0]):
        #     labels.append(label_value.astype("int"))
        
    except:
        print(f"face error: {feature_file} not exist")
        continue
    
    pose_json_data = []
    # Load pose data of a video
    for file in json_files: # for a single frame
        file_path = os.path.join(json_directory, file)
        try:
            with open(file_path, 'r') as f:
                data = json.load(f)
            
            # 漏检测 只有1人的情况
            if len(data['people']) != 2:
                row0 = data['people'][0]['pose_keypoints_2d']
                # 插入face_id
                row0.insert(0,0)
                 # 插入frame
                frame_id = int(file.split('_')[-2])//5 # 恢复frame id
                row0.insert(0,frame_id)
                pose_json_data.append(row0)

                continue
            
            row0 = data['people'][0]['pose_keypoints_2d']
            row1 = data['people'][1]['pose_keypoints_2d']


            # face_id对应 x较小 标记为1，x较大 标记为0 
            if(row0[0]<row1[0]):
                row0,row1 = row1,row0 # swap

            # 插入face_id
            row0.insert(0,0)
            row1.insert(0,1)

            # 插入frame
            frame_id = int(file.split('_')[-2])//5 + 1 # 恢复frame id 
            row0.insert(0,frame_id)
            row1.insert(0,frame_id)


            # 重复frame标签
            # labels.append(label_value.astype("int"))
            pose_json_data.append(row0)
            pose_json_data.append(row1)

        except:
            print(f"pose error: {file} not exist")
            continue
    features_pose.append(np.array(pose_json_data))

    # print(len(face_csv_data))
    # print(len(pose_json_data))
    
    # 将两个数组合并为一个数组

    face_array = np.array(features_face)
    pose_array = np.array(features_pose) 

    for row in pose_array:
        key = tuple(row[:2])
        match = face_array[face_array[:, :2] == key]
        if match.size > 0:
            features.append(np.concatenate((row, match[0, 2:]), axis=None))
    # combined = np.array(features)






# 将特征和标签转换为numpy数组
# X1 = np.concatenate(features_face, axis=0)
# X2 = np.concatenate(features_pose, axis=0)

X = np.concatenate(features, axis=0)
y = np.array(labels)

print(f"X:{len(X)}, y:{len(y)}")



/tmp/ipykernel_159773/2986659071.py:122: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  match = face_array[face_array[:, :2] == key]
/tmp/ipykernel_159773/2986659071.py:117: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  face_array = np.array(features_face)
/tmp/ipykernel_159773/2986659071.py:118: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  pose_array = np.array(features_pose)


IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [4]:
print(f"X:{len(X)}, y:{len(y)}")
print(f"X:{len(X2)}, y:{len(y)}")

X:39129, y:0
X:51216, y:0


In [9]:
X[2,:2]

array([2., 0.])

In [10]:
X2[2,:2]

array([1., 0.])